In [34]:
import faiss
import pandas as pd
import os
from tqdm import tqdm

In [2]:
df1 = pd.read_parquet("./wikipedia stem/0_to_25000.parquet")
df1.head()

,text,url,title
0,"Becurtovirus is a genus of viruses, in the fam...",https://en.wikipedia.org/wiki/Becurtovirus,Becurtovirus
1,Cyprinivirus is a genus of viruses in the orde...,https://en.wikipedia.org/wiki/Cyprinivirus,Cyprinivirus
2,"Glossinavirus is a genus of viruses, in the fa...",https://en.wikipedia.org/wiki/Glossinavirus,Glossinavirus
3,"Ichtadenovirus is a genus of viruses, in the f...",https://en.wikipedia.org/wiki/Ichtadenovirus,Ichtadenovirus
4,"Lambdatorquevirus is a genus of viruses, in th...",https://en.wikipedia.org/wiki/Lambdatorquevirus,Lambdatorquevirus


In [7]:
df1['text'].apply(lambda x: len(x.split(' '))).max()

20829

In [8]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer

model_name = 'sentence-transformers/all-MiniLM-L12-v2'
model = SentenceTransformer(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

c:\Users\86183\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\86183\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [32]:
def split_text_into_segments(text, max_length, stride, category):
    tokens = tokenizer.tokenize(text)
    segments = []
    categories = []

    start = 0
    while start < len(tokens):
        if start + max_length > len(tokens):
            segment_tokens = tokens[start:]
            segment = tokenizer.convert_tokens_to_string(segment_tokens)
            segments.append(segment)
            categories.append(category)
            break
        else:
            end = start + max_length
            segment_tokens = tokens[start:end]
            segment = tokenizer.convert_tokens_to_string(segment_tokens)
            segments.append(segment)
            categories.append(category)
            start = end - stride

    return segments, categories

In [21]:
long_text = "I love United States. This is a great country."
print(tokenizer.tokenize(long_text))
print(tokenizer.convert_tokens_to_string(tokenizer.tokenize(long_text)))

['i', 'love', 'united', 'states', '.', 'this', 'is', 'a', 'great', 'country', '.']
i love united states. this is a great country.


In [29]:
long_text = df1.loc[0, "text"]
stride = 64 
max_length = 384 

segments, categories = split_text_into_segments(long_text, max_length, stride)
print(segments)
print(len(segments))

['becurtovirus is a genus of viruses, in the family geminiviridae. dicotyledonous plants serve as natural hosts. there are three species in this genus. taxonomy the following three species are assigned to the genus : beet curly top iran virus exomis microphylla latent virus spinach curly top arizona virus structure viruses in becurtovirus are non - enveloped, with icosahedral geometries, and t = 1 symmetry. genomes are circular and non - segmented, around 3. 0kb in length. life cycle viral replication is nuclear. entry into the host cell is achieved by penetration into the host cell. replication follows the ssdna rolling circle model. dna - templated transcription is the method of transcription. the virus exits the host cell by nuclear pore export, and tubule - guided viral movement. dicotyledonous plants serve as the natural host. references external links viralzone : becurtovirus ictv geminiviridae virus genera']
1


In [35]:
chunked_context = []
corresponding_categories = []

for filename in os.listdir("./wikipedia stem/"):
    filepath = os.path.join("./wikipedia stem/", filename)
    print(f"Splitting file {filepath}")
    df = pd.read_parquet(filepath)

    for index, row in tqdm(df.iterrows(), total=len(df)):
        context = row["text"]
        category = row["title"]
        max_length = 384
        stride = 128

        segments, categories = split_text_into_segments(context, max_length, stride, category)
        chunked_context.extend(segments)
        corresponding_categories.extend(categories)

print(len(chunked_context), len(corresponding_categories))

100%|██████████| 25000/25000 [02:28<00:00, 168.70it/s]

541776 541776


In [36]:
df = pd.DataFrame({"context": chunked_context, "title": corresponding_categories})
df.shape

(541776, 2)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541776 entries, 0 to 541775
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   context  541776 non-null  object
 1   title    541776 non-null  object
dtypes: object(2)
memory usage: 8.3+ MB
